In [1]:
import pandas as pd           
import praw                   
import re                     
import datetime as dt
import seaborn as sns
import requests
import json
import sys
import time
## acknowledgements
'''
https://stackoverflow.com/questions/48358837/pulling-reddit-comments-using-python-praw-and-creating-a-dataframe-with-the-resu
https://www.reddit.com/r/redditdev/comments/2e2q2l/praw_downvote_count_always_zero/
https://towardsdatascience.com/an-easy-tutorial-about-sentiment-analysis-with-deep-learning-and-keras-2bf52b9cba91

For navigating pushshift: https://github.com/Watchful1/Sketchpad/blob/master/postDownloader.py

# traffic = reddit.subreddit(subreddit).traffic() is not available to us, sadly.
'''

with open("API.env") as file:
    exec(file.read())

reddit = praw.Reddit(
  client_id = client_id,
  client_secret = client_secret,
  user_agent = user_agent
)

'''
Some helper functions for the reddit API.
'''

def extract_num_rewards(awardings_data):
    return sum( x["count"] for x in awardings_data)

def extract_data(submission, comments = True):
    postlist = []

    # extracts top level comments
    if comments:
        submission.comments.replace_more(limit=0)
        for comment in submission.comments: 
            post = {} # put this here
            post['author'] = comment.author
            post['body'] = comment.body
            post['awards'] = comment.all_awardings
            post['score'] = comment.score
            post['parent_id'] = comment.parent_id
            post['id'] = comment.id

            postlist.append(post)

    content = {
    "title" : submission.title,
    "self" : submission.is_self,
    "text" : submission.selftext,
    "comments" : postlist,
    "author" : submission.author,
    "name" : submission.name,
    "upvote_ratio" : submission.upvote_ratio,
    "ups" : submission.score, #this is the same as submission.ups,
    "downs" : None,
    "awarders" : submission.awarders, 
    "awards" : submission.all_awardings,
    "total_awards" : None,
    "url" : submission.url # Only relevent if not a self post
    }
    
    content["total_awards"] = extract_num_rewards(content["awards"])
    content["downs"] = (1 - content["upvote_ratio"]) * content["ups"] / content["upvote_ratio"]
    return content

Version 7.0.0 of praw is outdated. Version 7.2.0 was released Wednesday February 24, 2021.


In [126]:
df = pd.read_pickle("may_wsb.pkl").sort_values(by = "score", ascending = False)
df

,title,id,created_utc,author_fullname,is_self,score,upvote_ratio,total_awards_received,selftext,url
2493,HOLDGME,n5vldf,1620263260,t2_5syqfl4h,False,59820,0.86,199,,https://i.redd.it/jwhf5yp0hex61.jpg
3323,Monster Crash,n5lboc,1620235294,t2_1jew76qz,False,53509,0.89,362,,https://v.redd.it/shekh7yq5cx61
3844,VLAD,n5g278,1620221695,t2_1jdyhbma,False,50248,0.90,176,,https://i.redd.it/yos8j48d1bx61.jpg
7923,WSB analyzes some charts,n3dujk,1619983656,t2_9js0dd4x,False,46432,0.85,432,,https://v.redd.it/2qjyeacadrw61
1719,The Insurrection of the Apes,n63q7n,1620294990,t2_a2jxbzsv,False,40180,0.89,251,,https://v.redd.it/xl5x239t2hx61
...,...,...,...,...,...,...,...,...,...,...
1939,Delta Lackies lick my balls,n61av0,1620284196,t2_a5lvbr25,True,0,0.27,0,Posted a statement that was honest about the w...,https://www.reddit.com/r/wallstreetbets/commen...
1941,So SOS in one hour let's get it going boys. Th...,n61a6r,1620284116,t2_5mtbu1vc,True,0,0.46,0,,https://www.reddit.com/r/wallstreetbets/commen...
1942,Lady Apes Unite! Let's Talk About FEMALES &amp...,n61a5g,1620284112,t2_1zum4908,True,0,0.43,0,# This is going to be a DD-ish discussion (my...,https://www.reddit.com/r/wallstreetbets/commen...
1944,He will Yolo 10 k on if we can get this video ...,n619qt,1620284069,t2_298qvwuc,False,0,0.35,0,,https://youtu.be/jKAxJo9wq9g


In [119]:
df2 = pd.DataFrame(df.head(100), copy = True)

In [131]:
monitored_df = df # df # pd.DataFrame(df.head(50)) # I don't know why using just the head is creating all these issues...

def make_dict_series(length):
    as_list = []
    for i in range(length):
        as_list.append( {})

    return pd.Series(as_list)

monitored_df["scores_timeseries"] = make_dict_series(len(monitored_df))
monitored_df["ratio_timeseries"] = make_dict_series(len(monitored_df))
monitored_df["total_awards_received_timeseries"] = make_dict_series(len(monitored_df))
#monitored_df = monitored_df.head(100)
pd.options.display.max_colwidth = 1000
monitored_df[["title", "url"]]

,title,url
2493,HOLDGME,https://i.redd.it/jwhf5yp0hex61.jpg
3323,Monster Crash,https://v.redd.it/shekh7yq5cx61
3844,VLAD,https://i.redd.it/yos8j48d1bx61.jpg
7923,WSB analyzes some charts,https://v.redd.it/2qjyeacadrw61
1719,The Insurrection of the Apes,https://v.redd.it/xl5x239t2hx61
...,...,...
1939,Delta Lackies lick my balls,https://www.reddit.com/r/wallstreetbets/comments/n61av0/delta_lackies_lick_my_balls/
1941,So SOS in one hour let's get it going boys. Those earnings are fire,https://www.reddit.com/r/wallstreetbets/comments/n61a6r/so_sos_in_one_hour_lets_get_it_going_boys_those/
1942,Lady Apes Unite! Let's Talk About FEMALES &amp; Retail Investing!,https://www.reddit.com/r/wallstreetbets/comments/n61a5g/lady_apes_unite_lets_talk_about_females_retail/
1944,He will Yolo 10 k on if we can get this video to reach 100k likes we half way there do your thing boys,https://youtu.be/jKAxJo9wq9g


In [123]:
''' 
Code to get time series of scores/ratios over time, by taking snapshots.
'''

for s in range(3):
    current_time = int(dt.datetime.now().replace(tzinfo=dt.timezone.utc).timestamp())
    scores = []
    total_awards_received = []
    ratios = []

    # Based on this: https://www.reddit.com/r/redditdev/comments/aoe4pk/praw_getting_multiple_submissions_using_by_id/
    ids2 = [i if i.startswith('t3_') else f't3_{i}' for i in list(monitored_df.id)]
    for submission in reddit.info(ids2): # Makes a single call to the PRAW API, much faster than doing them one by one.
        scores.append(submission.score)
        total_awards_received.append(extract_num_rewards(submission.all_awardings))
        ratios.append( submission.upvote_ratio)

    for i in range(len(monitored_df)):
        monitored_df['scores_timeseries'].iloc[i][current_time] = scores[i]
        monitored_df['ratio_timeseries'].iloc[i][current_time] = ratios[i]
        monitored_df['total_awards_received_timeseries'].iloc[i][current_time] = total_awards_received[i]

    print(f"updated at time {current_time}")
    time.sleep(3600)
        


updated at time 1620356536
updated at time 1620356545
updated at time 1620356553
